In [1]:
import requests
import pandas as pd
import numpy as np
from lxml import etree
from geopy.geocoders import Nominatim
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import json

print('Libraries imported.')

Libraries imported.


In [3]:
"""Needed for first time only"""
tbo_df = pd.read_excel('Toronto Borough.xlsx')
tbo_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Port Union
5,M1E,Scarborough,Guildwood
6,M1E,Scarborough,Morningside
7,M1E,Scarborough,West Hill
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


In [3]:
"""Needed for first time only"""
toronto = 'Toronto'
latList = []
longList = []
borList = []
neiList = []

In [ ]:
"""Needed for first time only"""
from opencage.geocoder import OpenCageGeocode

key = 'geocode key'
geocoder = OpenCageGeocode(key)

failed_rows = []
for index, row in tbo_df.iterrows():
    query = row['Neighbourhood'] + ", " + row['Borough'] + ", "  + "Toronto, Canada"
    #query = u'Bosutska ulica 10, Trnje, Zagreb, Croatia'
    results = geocoder.geocode(query)
    if(index % 10 == 0):
        print(index)
    if(results is None):
        failed_rows.append(results)
    else:
        latList.append(results[0]['geometry']['lat'])
        longList.append(results[0]['geometry']['lng'])
        neiList.append(row['Neighbourhood'])
        borList.append(row['Borough'])
lat_def = pd.DataFrame({ 'Bor' : borList, 'Nei': neiList, 'lat': latList, 'log': longList})
lat_def.to_csv(path_or_buf='Toronto_lat_log.csv')
lat_def.head()

In [4]:
lat_def = pd.read_csv('Toronto_lat_log.csv', index_col=0)
lat_def.head()

,Bor,Nei,lat,log
0,Scarborough,Rouge,43.804930,-79.165837
1,Scarborough,Malvern,43.809196,-79.221701
2,Scarborough,Highland Creek,43.790117,-79.173334
3,Scarborough,Rouge Hill,43.780271,-79.130499
4,Scarborough,Port Union,43.775504,-79.134977


In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(lat_def['Bor'].unique()),
        lat_def.shape[0]
    )
)

The dataframe has 11 boroughs and 212 neighborhoods.


get the latitude and longitude values of Tonoronto, CA

In [6]:
from opencage.geocoder import OpenCageGeocode

key = 'geocode key'
geocoder = OpenCageGeocode(key)

address = 'Toronto, CA'
results = geocoder.geocode(address)

latitude = results[0]['geometry']['lat']
longitude = results[0]['geometry']['lng']
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [4]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(lat_def['lat'], lat_def['log'], lat_def['Bor'], lat_def['Nei']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

NameError: name 'latitude' is not defined

In [8]:
lat_def['Bor'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [9]:
len(lat_def)

212

### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'client id' # your Foursquare ID
CLIENT_SECRET = 'secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Segment and cluster only the neighborhoods in Etobicoke. So let's slice the original dataframe and create a new dataframe

In [11]:
etk_data = lat_def[lat_def['Bor'] == 'Etobicoke'].reset_index(drop=True)
etk_data.head()

,Bor,Nei,lat,log
0,Etobicoke,Humber Bay Shores,43.700110,-79.416300
1,Etobicoke,Mimico South,43.603656,-79.493178
2,Etobicoke,New Toronto,43.600763,-79.505264
3,Etobicoke,Alderwood,43.601717,-79.545232
4,Etobicoke,Long Branch,43.593075,-79.541212


Let's get the geographical coordinates of Etobicoke

In [12]:
address = 'Etobicoke, CA'
results = geocoder.geocode(address)

latitude = results[0]['geometry']['lat']
longitude = results[0]['geometry']['lng']
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.6714591, -79.5524921.


let's visualizat Etobicoke the neighborhoods in it.

In [13]:
# create map of Etobicoke using latitude and longitude values
map_etk = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(etk_data['lat'], etk_data['log'], etk_data['Nei']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etk)  
    
map_etk

Get the neighborhood's name.

In [14]:
etk_data.loc[0, 'Nei']

'Humber Bay Shores'

Get the neighborhood's latitude and longitude values.

In [15]:
neighborhood_latitude = etk_data.loc[0, 'lat'] # neighborhood latitude value
neighborhood_longitude = etk_data.loc[0, 'log'] # neighborhood longitude value

neighborhood_name = etk_data.loc[0, 'Nei'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Humber Bay Shores are 43.700109999999995, -79.4163.


Now, let's get the top 100 venues that are in Humber Bay Shores within a radius of 500 meters.

In [ ]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

In [17]:
"""Run on first time only"""
"""Send the GET request and examine the resutls"""
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c923a0cdb04f53b16f12c58'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b896ec4f964a520bd3532e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16d941735',
         'name': 'Café',
         'pluralName': 'Cafés',
         'primary': True,
         'shortName': 'Café'}],
       'id': '4b896ec4f964a520bd3532e3',
       'location': {'address': '532 Eglinton Ave. W',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Heddington Ave.',
        'distance': 407,
        'formattedAddress': ['532 Eglinton Ave. W (at Heddington Ave.)',
         'Toronto ON M5N 1B4',
         'Canada'],
        'labeledLatLngs': [{'l

In [21]:
"""Run on first time only"""
with open('url_json_data.json', 'w') as outfile:
    json.dump(results, outfile, indent=4)

In [22]:
with open('url_json_data.json') as json_file:  
    results = json.load(json_file)
results

{'meta': {'code': 200, 'requestId': '5c923a0cdb04f53b16f12c58'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b896ec4f964a520bd3532e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16d941735',
         'name': 'Café',
         'pluralName': 'Cafés',
         'primary': True,
         'shortName': 'Café'}],
       'id': '4b896ec4f964a520bd3532e3',
       'location': {'address': '532 Eglinton Ave. W',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Heddington Ave.',
        'distance': 407,
        'formattedAddress': ['532 Eglinton Ave. W (at Heddington Ave.)',
         'Toronto ON M5N 1B4',
         'Canada'],
        'labeledLatLngs': [{'l

Now we are ready to clean the json and structure it into a pandas dataframe.

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Hotel Gelato,Café,43.703478,-79.414311
1,The Abbot,Gastropub,43.703688,-79.413485
2,The Mad Bean Coffee House,Coffee Shop,43.703529,-79.413698
3,7 Numbers,Italian Restaurant,43.703630,-79.413724
4,Ferraro,Italian Restaurant,43.703655,-79.413167


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Etobicoke

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
"""Run first time only"""
etk_venues = getNearbyVenues(names=etk_data['Nei'],
                                   latitudes=etk_data['lat'],
                                   longitudes=etk_data['log']
                                  )


Humber Bay Shores
Mimico South
New Toronto
Alderwood
Long Branch
The Kingsway
Montgomery Road
Old Mill North
Humber Bay
King's Mill Park
Kingsway Park South East
Mimico NE
Old Mill South
The Queensway East
Royal York South East
Sunnylea
Kingsway Park South West
Mimico NW
The Queensway West
Royal York South West
South of Bloor
Islington Avenue
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Westmount
Kingsview Village
Martin Grove Gardens
Richview Gardens
St. Phillips
Albion Gardens
Beaumond Heights
Humbergate
Jamestown
Mount Olive
Silverstone
South Steeles
Thistletown
Northwest


In [30]:
type(etk_venues)

pandas.core.frame.DataFrame

In [31]:
"""Run first time only"""
etk_venues.to_excel("etk_venues.xlsx")

In [32]:
etk_venues = pd.read_excel('etk_venues.xlsx')
etk_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Humber Bay Shores,43.70011,-79.4163,Hotel Gelato,43.703478,-79.414311,Café
1,Humber Bay Shores,43.70011,-79.4163,The Abbot,43.703688,-79.413485,Gastropub
2,Humber Bay Shores,43.70011,-79.4163,The Mad Bean Coffee House,43.703529,-79.413698,Coffee Shop
3,Humber Bay Shores,43.70011,-79.4163,7 Numbers,43.703630,-79.413724,Italian Restaurant
4,Humber Bay Shores,43.70011,-79.4163,Ferraro,43.703655,-79.413167,Italian Restaurant


In [34]:
"""et's check how many venues were returned for each neighborhood"""
etk_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Albion Gardens,16,16,16,16,16,16
Alderwood,9,9,9,9,9,9
Beaumond Heights,16,16,16,16,16,16
Bloordale Gardens,8,8,8,8,8,8
Cloverdale,26,26,26,26,26,26
Eringate,1,1,1,1,1,1
Humber Bay,3,3,3,3,3,3
Humber Bay Shores,16,16,16,16,16,16
Humbergate,8,8,8,8,8,8


In [36]:
"""Let's find out how many unique categories can be curated from all the returned venues"""
print('There are {} uniques categories.'.format(len(etk_venues['Venue Category'].unique())))

There are 97 uniques categories.


## 3. Analyze Each Neighborhood

In [37]:
# one hot encoding
etk_onehot = pd.get_dummies(etk_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etk_onehot['Neighborhood'] = etk_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [etk_onehot.columns[-1]] + list(etk_onehot.columns[:-1])
etk_onehot = etk_onehot[fixed_columns]

etk_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,...,Spa,Sporting Goods Shop,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant,Women's Store,Yoga Studio
0,Humber Bay Shores,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Humber Bay Shores,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Humber Bay Shores,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Humber Bay Shores,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Humber Bay Shores,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
etk_onehot.shape

(457, 98)

In [40]:
etk_grouped = etk_onehot.groupby('Neighborhood').mean().reset_index()
etk_grouped

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,...,Spa,Sporting Goods Shop,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant,Women's Store,Yoga Studio
0,Albion Gardens,0.000000,0.0,0.125,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.062500,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
1,Alderwood,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
2,Beaumond Heights,0.000000,0.0,0.125,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.062500,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
3,Bloordale Gardens,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
4,Cloverdale,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.038462,0.000000,0.00000,0.00000,0.000000,0.038462,0.000000,0.000000
5,Eringate,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
6,Humber Bay,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
7,Humber Bay Shores,0.000000,0.0,0.125,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.062500,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
8,Humbergate,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
9,Islington,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.200000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000


In [42]:
"""Let's print each neighborhood along with the top 5 most common venues"""
num_top_venues = 5

for hood in etk_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = etk_grouped[etk_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albion Gardens----
                venue  freq
0         Coffee Shop  0.12
1    Asian Restaurant  0.12
2  Italian Restaurant  0.12
3        Dance Studio  0.06
4  Chinese Restaurant  0.06


----Alderwood----
          venue  freq
0   Pizza Place  0.22
1           Gym  0.11
2  Skating Rink  0.11
3      Pharmacy  0.11
4          Pool  0.11


----Beaumond Heights----
                venue  freq
0         Coffee Shop  0.12
1    Asian Restaurant  0.12
2  Italian Restaurant  0.12
3        Dance Studio  0.06
4  Chinese Restaurant  0.06


----Bloordale Gardens----
               venue  freq
0  Convenience Store  0.25
1      Deli / Bodega  0.12
2       Intersection  0.12
3         Donut Shop  0.12
4        Coffee Shop  0.12


----Cloverdale----
                  venue  freq
0  Fast Food Restaurant  0.08
1           Coffee Shop  0.08
2            Restaurant  0.04
3     Electronics Store  0.04
4           Pizza Place  0.04


----Eringate----
                  venue  freq
0                  Par

----Westmount----
                  venue  freq
0           Pizza Place   0.5
1                  Park   0.5
2   American Restaurant   0.0
3  Gym / Fitness Center   0.0
4            Playground   0.0




In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
"""Now let's create the new dataframe and display the top 10 venues for each neighborhood."""
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = etk_grouped['Neighborhood']

for ind in np.arange(etk_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etk_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albion Gardens,Asian Restaurant,Coffee Shop,Italian Restaurant,Gastropub,Fruit & Vegetable Store,Dance Studio,Chinese Restaurant,Café,Japanese Restaurant,Gym / Fitness Center
1,Alderwood,Pizza Place,Gym,Skating Rink,Pool,Coffee Shop,Pub,Sandwich Place,Pharmacy,Flower Shop,Dessert Shop
2,Beaumond Heights,Asian Restaurant,Coffee Shop,Italian Restaurant,Gastropub,Fruit & Vegetable Store,Dance Studio,Chinese Restaurant,Café,Japanese Restaurant,Gym / Fitness Center
3,Bloordale Gardens,Convenience Store,Shipping Store,Deli / Bodega,Coffee Shop,Donut Shop,Intersection,Sandwich Place,Yoga Studio,Dessert Shop,Discount Store
4,Cloverdale,Coffee Shop,Fast Food Restaurant,Pharmacy,Record Shop,Pub,Café,Pizza Place,Pet Store,Clothing Store,Mobile Phone Shop


## 4. Cluster Neighborhoods

In [57]:
# set number of clusters
kclusters = 5

etk_grouped_clustering = etk_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etk_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 4, 1, 0, 0, 0], dtype=int32)

In [58]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

etk_merged = etk_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
etk_merged = etk_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Nei')

etk_merged.head() # check the last columns!

,Bor,Nei,lat,log,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Etobicoke,Humber Bay Shores,43.700110,-79.416300,0.0,Asian Restaurant,Coffee Shop,Italian Restaurant,Gastropub,Fruit & Vegetable Store,Dance Studio,Chinese Restaurant,Café,Japanese Restaurant,Gym / Fitness Center
1,Etobicoke,Mimico South,43.603656,-79.493178,0.0,Fast Food Restaurant,Gym / Fitness Center,Athletics & Sports,Sandwich Place,Bus Stop,Yoga Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run
2,Etobicoke,New Toronto,43.600763,-79.505264,0.0,Café,Pharmacy,Gym,Scenic Lookout,Fried Chicken Joint,Flower Shop,Indian Restaurant,Italian Restaurant,Mexican Restaurant,Dessert Shop
3,Etobicoke,Alderwood,43.601717,-79.545232,0.0,Pizza Place,Gym,Skating Rink,Pool,Coffee Shop,Pub,Sandwich Place,Pharmacy,Flower Shop,Dessert Shop
4,Etobicoke,Long Branch,43.593075,-79.541212,0.0,Seafood Restaurant,South American Restaurant,Convenience Store,Pharmacy,Coffee Shop,Restaurant,Café,Italian Restaurant,Burrito Place,Burger Joint


In [66]:
etk_merged['Cluster Labels'] = np.where(np.isnan(etk_merged['Cluster Labels']), 0, etk_merged['Cluster Labels'])

In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(etk_merged['lat'], etk_merged['log'], etk_merged['Nei'], etk_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [68]:
etk_merged.loc[etk_merged['Cluster Labels'] == 0, etk_merged.columns[[1] + list(range(5, etk_merged.shape[1]))]]

,Nei,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Humber Bay Shores,Asian Restaurant,Coffee Shop,Italian Restaurant,Gastropub,Fruit & Vegetable Store,Dance Studio,Chinese Restaurant,Café,Japanese Restaurant,Gym / Fitness Center
1,Mimico South,Fast Food Restaurant,Gym / Fitness Center,Athletics & Sports,Sandwich Place,Bus Stop,Yoga Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run
2,New Toronto,Café,Pharmacy,Gym,Scenic Lookout,Fried Chicken Joint,Flower Shop,Indian Restaurant,Italian Restaurant,Mexican Restaurant,Dessert Shop
3,Alderwood,Pizza Place,Gym,Skating Rink,Pool,Coffee Shop,Pub,Sandwich Place,Pharmacy,Flower Shop,Dessert Shop
4,Long Branch,Seafood Restaurant,South American Restaurant,Convenience Store,Pharmacy,Coffee Shop,Restaurant,Café,Italian Restaurant,Burrito Place,Burger Joint
5,The Kingsway,Coffee Shop,Dessert Shop,Breakfast Spot,Pub,Sushi Restaurant,Italian Restaurant,Liquor Store,Pool Hall,Pizza Place,Mobile Phone Shop
6,Montgomery Road,History Museum,Coffee Shop,Italian Restaurant,Yoga Studio,Fast Food Restaurant,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop
10,Kingsway Park South East,Coffee Shop,Dessert Shop,Breakfast Spot,Pub,Sushi Restaurant,Italian Restaurant,Liquor Store,Pool Hall,Pizza Place,Mobile Phone Shop
11,Mimico NE,Fast Food Restaurant,Gym / Fitness Center,Athletics & Sports,Sandwich Place,Bus Stop,Yoga Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run
13,The Queensway East,Gym,Garden Center,Coffee Shop,Sushi Restaurant,Eastern European Restaurant,Fast Food Restaurant,Deli / Bodega,Dessert Shop,Discount Store,Dog Run


In [69]:
etk_merged.loc[etk_merged['Cluster Labels'] == 1, etk_merged.columns[[1] + list(range(5, etk_merged.shape[1]))]]

,Nei,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Old Mill North,Park,American Restaurant,Spa,Metro Station,Fast Food Restaurant,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop
8,Humber Bay,Construction & Landscaping,Deli / Bodega,Park,Convenience Store,Dessert Shop,Discount Store,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store
9,King's Mill Park,Park,American Restaurant,Spa,Metro Station,River,Playground,Bakery,Dance Studio,Deli / Bodega,Dessert Shop
12,Old Mill South,Park,American Restaurant,Spa,Metro Station,Fast Food Restaurant,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop
21,Islington Avenue,Park,Moving Target,Baseball Field,Yoga Studio,Fast Food Restaurant,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop
29,Markland Wood,Golf Course,Piano Bar,Park,Baseball Field,Fast Food Restaurant,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop
30,Old Burnhamthorpe,Dog Run,Park,Flower Shop,Yoga Studio,Fast Food Restaurant,Deli / Bodega,Dessert Shop,Discount Store,Donut Shop,Eastern European Restaurant
31,Westmount,Pizza Place,Park,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop,Eastern European Restaurant


In [70]:
etk_merged.loc[etk_merged['Cluster Labels'] == 2, etk_merged.columns[[1] + list(range(5, etk_merged.shape[1]))]]

,Nei,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Martin Grove,Playground,Women's Store,Garden,Supermarket,Coffee Shop,Flower Shop,Event Space,Dance Studio,Deli / Bodega,Dessert Shop
39,Jamestown,Playground,Women's Store,Garden,Supermarket,Coffee Shop,Flower Shop,Event Space,Dance Studio,Deli / Bodega,Dessert Shop
40,Mount Olive,Playground,Women's Store,Garden,Supermarket,Coffee Shop,Flower Shop,Event Space,Dance Studio,Deli / Bodega,Dessert Shop
44,Northwest,Playground,Women's Store,Garden,Supermarket,Coffee Shop,Flower Shop,Event Space,Dance Studio,Deli / Bodega,Dessert Shop


In [71]:
etk_merged.loc[etk_merged['Cluster Labels'] == 3, etk_merged.columns[[1] + list(range(5, etk_merged.shape[1]))]]

,Nei,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Richview Gardens,Intersection,Electronics Store,Yoga Studio,Fish & Chips Shop,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop,Eastern European Restaurant


In [72]:
etk_merged.loc[etk_merged['Cluster Labels'] == 4, etk_merged.columns[[1] + list(range(5, etk_merged.shape[1]))]]

,Nei,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Eringate,Park,Yoga Studio,Convenience Store,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store
32,Kingsview Village,Park,Yoga Studio,Convenience Store,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store
